# Optimal flow problem
## 1) Solve the SDP relaxation

In [20]:
import numpy as np
from matplotlib import image
from matplotlib import pyplot
from numpy import random as rnd
import cvxpy as cp
import matplotlib.pyplot as plt
from mat4py import loadmat

IEEE_9 = loadmat('Project4_Questions/IEEE9data.mat')

In [44]:
# Number of buses
N = 9

# Load constants
G = np.asarray(IEEE_9['data']['G'])
B = np.asarray(IEEE_9['data']['B'])
p_d = np.asarray(IEEE_9['data']['p']['d'])
q_d = np.asarray(IEEE_9['data']['q']['d'])
p_g_min = np.asarray(IEEE_9['data']['p']['min'])
p_g_max = np.asarray(IEEE_9['data']['p']['max'])
q_g_min = np.asarray(IEEE_9['data']['q']['min'])
q_g_max = np.asarray(IEEE_9['data']['q']['max'])
c = np.asarray(IEEE_9['data']['c'])
v_min = np.asarray(IEEE_9['data']['v']['min'])
v_max = np.asarray(IEEE_9['data']['v']['max'])

# Variables
p_g = cp.Variable((N,1))
q_g = cp.Variable((N,1))
E, F = cp.Variable((N,N),symmetric=True), cp.Variable((N,N),symmetric=True)
H = cp.Variable((N,N))

# Constraints
constraints = []
for i in range(N):
    constraints.append(G[i,:,np.newaxis].T@(E[i,:,np.newaxis] + F[i,:,np.newaxis]) -
                       B[i,:,np.newaxis].T@(H[i,:,np.newaxis] - H[:,i,np.newaxis].T) == p_g - p_d)
    constraints.append(-B[i,:,np.newaxis].T@(E[i,:,np.newaxis] + F[i,:,np.newaxis]) -
                       G[i,:,np.newaxis].T@(H[i,:,np.newaxis] - H[:,i,np.newaxis].T) == q_g - q_d)
constraints.extend([p_g_min <= p_g , p_g <= p_g_max])
constraints.extend([q_g_min <= q_g , q_g <= q_g_max])
constraints.extend([cp.square(v_min) <= cp.diag(E + F)[:,np.newaxis] ,
                    cp.square(v_max) >= cp.diag(E + F)[:,np.newaxis]])
constraints.append(cp.bmat([[E,H],[H.T,F]]) >= 0)

# Loss
objective = cp.Minimize(c.T@p_g)

prob = cp.Problem(objective, constraints)
prob.solve(solver=cp.MOSEK)

inf